# Data Splitting
* 加上負樣本

## Import 

In [20]:
import pandas as pd
import json
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import pickle
import math 

In [57]:
df = pd.read_pickle('/home/adam/Steph_C/my_thesis/data/FE_by_postoal_code_without_review.pkl')

FileNotFoundError: [Errno 2] No such file or directory: '/home/adam/Steph_C/my_thesis/data/ORI_by_postoal_code_without_review.pkl'

In [22]:
df.shape

(1425, 26)

In [23]:
df.columns

Index(['business_id', 'stars_x', 'useful', 'funny', 'cool', 'text', 'date',
       'name', 'address', 'city', 'state', 'postal_code', 'latitude',
       'longitude', 'stars_y', 'review_count', 'is_open', 'attributes',
       'categories', 'hours', 'density', 'entropy', 'competitiveness',
       'area_pop', 'accessibility', 'complementary'],
      dtype='object')

In [24]:
len(Counter(df.name))

565

## DF Construction
* add relevance

In [27]:
# sort by name and review count 
df = df.sort_values(['name', 'review_count'],
              ascending = [True, False]).reset_index(drop=True)

In [28]:
# create relevance score
cnt = 0
new_df = pd.DataFrame()
for i in Counter(df.name):
    tmp = df[df.name==i].reset_index(drop=True)
    tmp['relevance']=''
    score = 6
    for j in range(len(tmp)):
        tmp['relevance'][j]=score
        score -=1
    new_df = pd.concat([new_df,tmp])

# check the shape
if new_df.shape[0] != df.shape[0]:
    print(f'There is a mistake creating the relevance score')
else:
    print(f'Relevance score added')

/home/adam/.local/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


Relevance score added


In [29]:
new_df = new_df.drop(columns=[ 'business_id','city','state', 'review_count','categories','stars_x', 'useful', 'funny', 'cool', 'text',
       'date','address','is_open', 'stars_y','attributes','hours'])

## Create Dataset

In [30]:
new_df.columns

Index(['name', 'postal_code', 'latitude', 'longitude', 'density', 'entropy',
       'competitiveness', 'area_pop', 'accessibility', 'complementary',
       'relevance'],
      dtype='object')

In [31]:
# postal code and feature dict
postal_code_feature_dict = {}

for postal in Counter(df.postal_code):
    postal_code_feature_dict[postal]={}
    postal_code_feature_dict[postal]['density'] = df.loc[df['postal_code']==postal]['density'].iloc[0]
    postal_code_feature_dict[postal]['entropy'] = df.loc[df['postal_code']==postal]['entropy'].iloc[0]
    postal_code_feature_dict[postal]['competitiveness'] = df.loc[df['postal_code']==postal]['competitiveness'].iloc[0]
    postal_code_feature_dict[postal]['area_pop'] = df.loc[df['postal_code']==postal]['area_pop'].iloc[0]
    postal_code_feature_dict[postal]['accessibility'] = df.loc[df['postal_code']==postal]['accessibility'].iloc[0]
    postal_code_feature_dict[postal]['complementary'] = df.loc[df['postal_code']==postal]['complementary'].iloc[0]

pd.DataFrame(postal_code_feature_dict).T.reset_index().rename(columns={'index': 'postal_code'})

,postal_code,density,entropy,competitiveness,area_pop,accessibility,complementary
0,46225,5.0,1.609438,-0.200000,1890.0,0.0,0.0
1,46037,5.0,1.609438,-0.200000,1972.0,0.0,0.0
2,33618,12.0,2.484907,-0.083333,3630.0,0.0,0.0
3,33609,11.0,2.397895,-0.090909,4551.0,0.0,0.0
4,63126,3.0,1.098612,-0.333333,492.0,0.0,0.0
...,...,...,...,...,...,...,...
351,08055,1.0,-0.000000,-1.000000,160.0,0.0,0.0
352,08057,1.0,-0.000000,-1.000000,710.0,0.0,0.0
353,18915,1.0,-0.000000,-1.000000,111.0,0.0,0.0
354,33771,1.0,-0.000000,-1.000000,572.0,0.0,0.0


In [32]:
# POINTWISE
# split train test 
# 拿每一個餐廳一半的分店當作 testing set (雖然大部分只有兩家分店)

cnt = 0
train_df = pd.DataFrame()
test_df = pd.DataFrame()

for i in Counter(df.name):
    
    tmp = new_df[new_df.name==i]
    if len(tmp) %2 ==0:
        row_count = int(len(tmp)/2)
    else:
        row_count = int(len(tmp)/2)+1
    tmp_train = tmp.iloc[:row_count,:]
    tmp_test = tmp.iloc[row_count:,:]
    
    other_places = pd.DataFrame(postal_code_feature_dict).T.reset_index().rename(columns={'index': 'postal_code'})
    other_places = other_places.sort_values(['postal_code'],
              ascending = [True]).reset_index(drop=True)
    
    ############################## reconstruct ################################
    postal_codes_list = list(other_places.postal_code)
    
    for j in Counter(tmp_train.postal_code):
        postal_codes_list.remove(j)
    
    for j in Counter(tmp_test.postal_code):
        postal_codes_list.remove(j)

    ## train 
    for j in Counter(tmp_train.postal_code):
        cnt = 0
        concat_index =[]
        for k in range(len(other_places)):
            if cnt <5:
                if other_places.postal_code[k] in postal_codes_list and abs(int(other_places.postal_code[k])-int(j)) <= 10000:
                    postal_codes_list.remove(other_places.postal_code[k])
                    cnt+=1
                    concat_index.append(k)
        others = other_places.iloc[concat_index]
        others['relevance'] = 0
        others['name'] = i
        tmp_train = pd.concat([tmp_train ,others ])
    
    ## test 
    for j in Counter(tmp_test.postal_code):
        cnt = 0
        concat_index =[]
        for k in range(len(other_places)):
            if cnt <5:
                if other_places.postal_code[k] in postal_codes_list and abs(int(other_places.postal_code[k])-int(j)) <= 10000:
                    postal_codes_list.remove(other_places.postal_code[k])
                    cnt+=1
                    concat_index.append(k)
        others = other_places.iloc[concat_index]
        others['relevance'] = 0
        others['name'] = i
        tmp_test = pd.concat([tmp_test ,others ])
    
    train_df =  pd.concat([train_df ,tmp_train ])
    test_df =  pd.concat([test_df ,tmp_test ])
print(f'Finished constructing....  train : {len(Counter(train_df.name))} test : {len(Counter(test_df.name))}')

/home/adam/.local/lib/python3.7/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/adam/.local/lib/python3.7/site-packages/ipykernel_launcher.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/adam/.local/lib/python3.7/site-packages/ipykernel_launcher.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata

In [37]:
# for i in ['density', 'entropy', 'competitiveness','area_pop', 'accessibility','complementary','relevance']:
#     train_df[i] = train_df[i].astype('float')
#     test_df[i] = test_df[i].astype('float')
# train_df.to_pickle('/home/adam/Steph_C/my_thesis/data/Train_by_postoal_code_without_review_pointwise.pkl')
# test_df.to_pickle('/home/adam/Steph_C/my_thesis/data/Test_by_postoal_code_without_review_pointwise.pkl')

In [54]:
# PAIRWISE
# split train test 
# 拿每一個餐廳一半的分店當作 testing set (雖然大部分只有兩家分店)

cnt = 0
train_df = pd.DataFrame()
test_df = pd.DataFrame()

for i in Counter(df.name):
    tmp = new_df[new_df.name==i]
    if len(tmp) %2 ==0:
        row_count = int(len(tmp)/2)
    else:
        row_count = int(len(tmp)/2)+1
    tmp_train = tmp.iloc[:row_count,:]
    tmp_test = tmp.iloc[row_count:,:]
    
    other_places = pd.DataFrame(postal_code_feature_dict).T.reset_index().rename(columns={'index': 'postal_code'})
    other_places = other_places.sort_values(['postal_code'],
              ascending = [True]).reset_index(drop=True)
    
    ############################## reconstruct ################################
    postal_codes_list = list(other_places.postal_code)
    
    for j in Counter(tmp_train.postal_code):
        postal_codes_list.remove(j)
    
    for j in Counter(tmp_test.postal_code):
        postal_codes_list.remove(j)

    ## train 
    tmp_train = tmp_train.reset_index(drop=True)
    tmp_train['pair_relevance'] =''
    for j in range(len(tmp_train)):
        pair_list = []
        pair_list.append(tmp_train.postal_code[j])
        other_loc = [x for x in postal_codes_list if abs(int(x) - int(pair_list[0])) <=3000][0]
        pair_list.append(other_loc)
        tmp_train.postal_code[j] = pair_list
        tmp_train.pair_relevance[j] = [1,0]
        postal_codes_list.remove(other_loc)
    
    ## test 
    tmp_test = tmp_test.reset_index(drop=True)
    tmp_test['pair_relevance'] =''
    for j in range(len(tmp_test)):
        pair_list = []
        pair_list.append(tmp_test.postal_code[j])
        other_loc = [x for x in postal_codes_list if abs(int(x) - int(pair_list[0])) <=3000][0]
        pair_list.append(other_loc)
        tmp_test.postal_code[j] = pair_list
        tmp_test.pair_relevance[j] = [1,0]
        postal_codes_list.remove(other_loc)
    
    train_df =  pd.concat([train_df ,tmp_train ])
    test_df =  pd.concat([test_df ,tmp_test ])
print(f'Finished constructing....  train : {len(Counter(train_df.name))} test : {len(Counter(test_df.name))}')

/home/adam/.local/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/adam/.local/lib/python3.7/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/adam/.local/lib/python3.7/site-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/adam/.local/lib/python3.7/site-packages/ipykernel_launcher.py:52: SettingWithCopyWarn

Finished constructing....  train : 565 test : 565


In [56]:
for i in ['density', 'entropy', 'competitiveness','area_pop', 'accessibility','complementary','relevance']:
    train_df[i] = train_df[i].astype('float')
    test_df[i] = test_df[i].astype('float')
train_df.to_pickle('/home/adam/Steph_C/my_thesis/data/Train_by_postoal_code_without_review_pairwise.pkl')
test_df.to_pickle('/home/adam/Steph_C/my_thesis/data/Test_by_postoal_code_without_review_pairwise.pkl')